## Building Predictive Models

In [1]:
import pandas as pd
import numpy as np
import os

### Import Data

In [2]:
#set the path of the processed data
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path,'train.csv')
test_file_path = os.path.join(processed_data_path,'test.csv')

In [3]:
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [ ]:
train_df.info()

In [ ]:
test_df.info()

### Data Preparation

In [4]:
X = train_df.loc[:,'Age':].as_matrix().astype('float')
y = train_df['Survived'].ravel()

/Users/jandiroquai/Durango/environment/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [5]:
print(X.shape, y.shape)

(891, 32) (891,)


In [6]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [7]:
#average survival in train and test
print('mean survival in train: {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test: 0.385


#### Check Scikit-Learn Version

In [8]:
import sklearn

In [9]:
sklearn.__version__

'0.21.3'

### Baseline Model

In [10]:
#import function
from sklearn.dummy import DummyClassifier

In [11]:
#create model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [ ]:
'''
Al usar la estrategia most_frequent el modelo dummy predecirá 
como resultado el valor más frecuente.  En este caso, cero
'''

In [12]:
#train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [ ]:
'''
la funcion .fit crea un modelo dummy
con input X_train y output y_train
'''

In [13]:
print('score for baseline model: {0:.2f}'.format(model_dummy.score(X_test, y_test)))

score for baseline model: 0.61


In [ ]:
'''
The previous number means that if it always uses 0 as a prediction value,
it will be right in 61 percent of the cases
'''

In [14]:
#performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [15]:
#accuracy score
print('Accuracy for baseline model: {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

Accuracy for baseline model: 0.61


In [16]:
#confusion matrix
print('Confusion matrix for baseline model \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

Confusion matrix for baseline model 
 [[110   0]
 [ 69   0]]


In [17]:
#precision and recall scores
print('Precision score for baseline model: {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('Recall score for baseline model: {0:2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

Precision score for baseline model: 0.00
Recall score for baseline model: 0.000000


/Users/jandiroquai/Durango/environment/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### First Kaggle Submission

In [ ]:
#Converting to the matrix
test_X = test_df.as_matrix().astype('float')

In [ ]:
#get predictions
predictions = model_dummy.predict(test_X)

In [ ]:
df_submission = pd.DataFrame({'PassangerId': test_df.index, 'Survived' : predictions})

In [ ]:
df_submission.head()

In [ ]:
submission_data_path = os.path.join(os.path.pardir,'data','external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')

In [ ]:
df_submission.to_csv(submission_file_path, index=False)

In [ ]:
#convert to a function
def get_submission_file(model, filename):
    test_X = test_df.as_matrix().astype('float')
    predictions = model.predict(test_X)
    df_submission = pd.DataFrame({'PassangerId': test_df.index, 'Survived' : predictions})
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path, filename)
    df_submission.to_csv(submission_file_path, index=False)

In [ ]:
#get submission file
get_submission_file(model_dummy, '01_dummy.csv')

In [ ]:
df_submission.info()

### Logistic Regression Model

In [18]:
#import function
from sklearn.linear_model import LogisticRegression

In [19]:
#create model
model_lr_1 = LogisticRegression(random_state=0)

In [20]:
#train model
model_lr_1.fit(X_train, y_train)

/Users/jandiroquai/Durango/environment/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
print('score for logistic regression -v1: {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistic regression -v1: 0.83


In [22]:
#calculate rest of performance metrics

In [23]:
#model coefficients
model_lr_1.coef_

array([[-0.02842268,  0.00455451, -0.50009089,  0.61781319, -0.81392331,
         0.12845079, -0.17281789, -0.39317834,  0.52159979,  1.09941224,
         0.40341217, -0.18345052, -0.30036043,  0.96533486,  0.48256744,
        -0.34483448,  0.28089598,  1.21761328,  0.56363966, -1.44586305,
         1.07245549, -0.11273708, -0.47293646,  0.16255648,  0.24716932,
         0.28009429,  0.41324772,  0.49183528,  0.46198829,  0.14924424,
         0.37283516,  0.73023265]])

In [ ]:
#get submission file
get_submission_file(model_lr_1, '02_lr.csv')

 ### Hyperparameter optimization

In [29]:
#base model
model_lr = LogisticRegression(random_state=0, solver='liblinear')

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1','l2']}
clf = GridSearchCV(model_lr, param_grid = parameters, cv = 3)

In [32]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [34]:
print('best score: {0:.2f}'.format(clf.best_score_))

best score: 0.83


In [35]:
#evaluate model
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test, y_test)))

score for logistic regression - version 2: 0.83


### Feature Normalizatio and Standardization

In [36]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Feature normalization

In [39]:
#feature normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [41]:
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [42]:
#normalize test data
X_test_scaled = scaler.transform(X_test)

#### Feature Standardization

In [43]:
#feature standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Create model after standardization

In [44]:
#base model
model_lr = LogisticRegression(random_state=0, solver='liblinear')
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1','l2']}
clf = GridSearchCV(model_lr, param_grid = parameters, cv = 3)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [46]:
clf.best_score_

0.8132022471910112

In [48]:
#evaluate model
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test_scaled, y_test)))

score for logistic regression - version 2: 0.84


### Model Persistance

In [49]:
# import pickle library
import pickle

In [50]:
#create file paths
model_file_path = os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir,'models','lr_scaler.pkl')

In [52]:
#open the files to write
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path,'wb')

In [54]:
#persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [55]:
#close the file
model_file_pickle.close()
scaler_file_pickle.close()

#### Load the persisted file

In [56]:
#open files in read mode
model_file_pickle = open(model_file_path, 'r')
scaler_file_pickle = open(scaler_file_path, 'r')
#load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
#close files
model_file_pickle.close()
scaler_file_pickle.close()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
clf_loaded

In [ ]:
scaler_loaded

In [ ]:
#transform the test data using loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
#calculate the score using the loaded model object
print('score for persisted logistic regression: {0:.2f}'.format(clf_loaded.score(X_test_scaled, y_test)))